# O problema da empresa

Nosso cliente é uma empresa de cartão de crédito. Eles nos trouxeram um dataset que inclui dados demográficos e dados financeiros recentes __(últimos seis meses)__ de uma amostra de __30.000 titulares__ de contas. Esses dados estão no nível de conta de crédito; em outras palavras, __há uma linha para cada conta__ (você deve sempre esclarecer qual é a definição de linha, em um dataset). As linhas são rotuladas de acordo com se no mês seguinte ao período de dados histórico de seis meses um proprietário de conta ficou inadimplente, ou seja, não fez o pagamento mínimo.

# Objetivo
Seu objetivo é desenvolver um modelo que preveja se __uma conta ficará inadimplente no próximo mês__, de acordo com dados demográficos e históricos. Posteriormente no livro, discutiremos a aplicação prática do modelo.

## Importando packeges

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px 
import seaborn as sns 


## Obtendo dados

In [3]:
df = pd.read_excel(r'../data/external/default_of_credit_card_clients__courseware_version_1_21_19.xls')
df.sample(3)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
17109,272780e2-0f01,170000,2,2,2,27,-1,-1,-1,0,...,23922,24399,24919,0,27107,1500,863,903,3000,1
10633,fcfdf19b-b7c0,50000,1,2,2,32,0,0,0,0,...,29626,26328,28475,1658,29000,1027,1300,2600,1280,0
1780,e6e8a31e-f123,450000,1,1,1,41,Not available,-1,-1,-2,...,0,461,-3,505,0,0,461,3,0,0


In [4]:
## Verificando quantidade de linhas e colunas
df.shape

(30000, 25)

# Etapas da exploração de dados
1. Saber quantas colunas os dados contêm.

    Podem ser de características, resposta ou metadados.
2. Quantas linhas (amostras).
3. Que tipos de características existem. Quais são categóricas e quais são numéricas?

    Os valores das características categóricas pertencem a classes discretas como “sim”, “não” ou “talvez”. Normalmente as características numéricas pertencem a uma escala numérica contínua, como as quantias em dólares.
4. Qual é a aparência dos dados segundo essas características.

    Para saber isso, você pode examinar o intervalo de valores em características numéricas ou a frequência de classes diferentes em características categóricas, por exemplo.
5. Há dados faltando?

## Verificando a integridade básica dos dados

In [7]:
# nomes das colunas
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

### Dicionário de dados
__LIMIT_BAL:__ Valor do crédito fornecido (em novos dólares taiwaneses (NT)) inclu-
sive o crédito do consumidor individual e familiar (complementar). SEX: Gênero (1 = masculino; 2 = feminino).
__Nota:__ Não usaremos os dados de gênero para tomar decisões de solvibilidade devido a considerações éticas.
__EDUCATION:__ Instrução (1 = pós-graduação; 2 = universidade; 3 = ensino médio; 4 = outros).
__MARRIAGE:__ Estado civil (1 = casado; 2 = solteiro; 3 = outros). AGE: Idade (ano).
__PAY_1–Pay_6:__ Registro de pagamentos passados. Pagamentos mensais passados, registrados de abril a setembro, são armazenados nessas colunas.
__PAY_1__ representa o status de reembolso em setembro; 
__PAY_2__ = status de reem- bolso em agosto; e assim por diante até __PAY_6__, que representa o status de reem- bolso em abril.

A __escala de medida__ do status de reembolso é a seguinte: 
    * -1= pagamento pontual; 
    * 1 = atraso de um mês no pagamento; 
    * 2 = atraso de dois meses no pagamento; 
    * e assim por diante até 8 = atraso de oito meses no pagamento; 
    * 9 = atraso de nove meses ou mais no pagamento.

__BILL_AMT1–BILL_AMT6:__ Valor da fatura (em novos dólares taiwaneses).

__BILL_AMT1__ representa o valor da fatura em setembro; 

__BILL_AMT2__ representa o valor da fatura em agosto; e assim por diante até:

__BILL_AMT7__, que representa o valor da fatura em abril.

__PAY_AMT1–PAY_AMT6__: Valor de pagamentos anteriores (novos dólares taiwaneses). 

__PAY_AMT1__ representa o valor pago em setembro; 
__PAY_AMT2__ representa o valor pago em agosto; e assim por diante até __PAY_AMT6__
, que representa o valor pago em abril.

In [8]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [10]:
df['ID'].nunique()

29687

In [11]:
df['ID'].value_counts()

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
                ..
2edeb3a6-d38d    1
27e11893-92e9    1
2b1c282b-441e    1
bd737997-0eb0    1
15d69f9f-5ad3    1
Name: ID, Length: 29687, dtype: int64

In [12]:
id_counts = df['ID'].value_counts()
id_counts.head()

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
Name: ID, dtype: int64

In [14]:
# Exiba o número de entradas duplicadas agrupadas executando outra conta- gem de valores:
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64